In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, fbeta_score, precision_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import multilabel_confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from skmultilearn.adapt import MLkNN
from sklearn.linear_model import LogisticRegression, Perceptron, RidgeClassifier, SGDClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.calibration import CalibratedClassifierCV
from sklearn.multioutput import MultiOutputClassifier


import pandas as pd
import json
import os
import re

import pickle
import nltk
from nltk.tokenize import word_tokenize

import gensim 
from gensim.models import Word2Vec
import gensim.downloader


import spacy
import matplotlib
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ast import literal_eval
from tqdm import tqdm

import sklearn.metrics
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('seaborn')

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [2]:
nltk.download('punkt')
tqdm.pandas()

[nltk_data] Downloading package punkt to /homes/lgf21/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Open Pickle without lemmatisation: 

with open('merged_data_no_duplicates.pickle', 'rb') as handle:
    (X_train_text, X_test_text, Y_train_tactic, Y_test_tactic, Y_train, Y_test) = pickle.load(handle)

In [4]:
X_train_text

0       Exploit Public-Facing Application - Enterprise...
1       Emergency Incident ResponseReport a Confirmed ...
2       Pass the Hash - Enterprise | MITRE ATT&CK\xe2\...
3       Extra Window Memory Injection - Enterprise | M...
4       Tropic Trooper Targets Taiwanese Government an...
                              ...                        
2142     Molerats Delivers  MALWARE_NAME  Backdoor to ...
2143     Transparent Tribe  Evolution analysis  part  ...
2144     WWW FIDELISSECURITY COM  Fidelis Cybersecurit...
2145     OilRig uses  MALWARE_NAME  IIS Backdoor on Ta...
2146     The OilRig Campaign  Attacks on Saudi Arabian...
Name: Text, Length: 2072, dtype: object

# Feature Extraction:

In [4]:
# call methods for feature extraction and evaluation:
import sys
sys.path.append('../src')

from methods import feature_extraction, evaluation

In [5]:
X_train, X_test = feature_extraction('TfIdfVectorizer', X_train_text, X_test_text)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [6]:
# Train and test: First delete techniques less than 9 
# We fix the random state to have the same dataset in our different tests@

sv_classifier = OneVsRestClassifier(LinearSVC(penalty = 'l2', loss = 'squared_hinge', dual = False, max_iter = 1000, class_weight = 'balanced', random_state=42), n_jobs = 1)
sv_classifier.fit(X_train, Y_train)


OneVsRestClassifier(estimator=LinearSVC(class_weight='balanced', dual=False,
                                        random_state=42),
                    n_jobs=1)

In [7]:
Y_pred = pd.DataFrame(sv_classifier.predict(X_test), columns=Y_test.columns)

In [ ]:
evaluation(Y_pred, Y_test)

In [ ]:
# Save as pickle technique performance: 

#with open('postprocessing.pickle', 'wb') as handle:
 #   pickle.dump(sv_classifier, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Open Tactic Performance: 

In [8]:
# Open Pickle: 

with open('tactic_MLP_TFIDF_model.pickle', 'rb') as handle:
    tactic_model = pickle.load(handle)

In [9]:
data = '../src/scraping/scraped_data/tactic_dataset.json'

In [10]:
with open(data) as file:
    tactic_data = json.load(file)

In [11]:
Y_test_pred_tactic = pd.DataFrame(tactic_model.predict(X_test), columns = Y_test_tactic.columns)

# Hanging Node:

In [12]:
# list of tactics and list of techniques:
from sklearn.preprocessing import MinMaxScaler

In [27]:
tactic_prob = pd.DataFrame(tactic_model.predict_proba(X_test), columns=Y_test_tactic.columns)

In [28]:
# if scaling at (-1,1) then threshold of .5 
# scaling determines improvement 
# (0-1) performs poorly with threshold of .5 

technique_prob = pd.DataFrame(MinMaxScaler((-1, 1)).fit_transform(sv_classifier.decision_function(X_test)), columns=Y_test.columns)

In [29]:
def hanging_node(i, tactic_proba, technique_proba,tactic_threshold=0.5, technique_threshold=0.5, a = 1,b = 0, c=1, d=0):
    assert a > technique_threshold and c > technique_threshold and b < tactic_threshold and d < tactic_threshold
    pred_tactics = [tactic for tactic in tactic_proba.columns if tactic_proba.iloc[i][tactic] > tactic_threshold]
    pred_techniques = [technique for technique in technique_proba.columns if technique_proba.iloc[i][technique] > technique_threshold]
    #return pred_tactics, pred_techniques
    print((pred_tactics, pred_techniques))
    for tactic in tactic_proba.columns:
        for technique in tactic_data[tactic]['Technique_ID'][0]:
            if technique in pred_techniques and tactic not in pred_tactics:
                if technique_proba.iloc[i][technique] > a and tactic_proba.iloc[i][tactic] > b:
                    pred_tactics.append(tactic)
                elif technique_proba.iloc[i][technique] < c and tactic_proba.iloc[i][tactic] < d:
                    pred_techniques.remove(technique)
    print(pred_tactics, pred_techniques)
    print('-----------------------------')
    return pred_tactics, pred_techniques
    

In [30]:
Y_test_tactics_hanging_node = []
Y_test_techniques_hanging_node = []
for i in range(len(X_test)):
    pred_tactics, pred_techniques = hanging_node(i, tactic_prob, technique_prob,tactic_threshold=0, technique_threshold=0.5,  a = 0.55, b = -0.9, c = 0.95, d = -0.30)
    Y_test_tactics_hanging_node.append([1 if tactic in pred_tactics else 0 for tactic in Y_test_tactic.columns])
    Y_test_techniques_hanging_node.append([1 if technique in pred_techniques else 0 for technique in Y_test.columns])

Y_test_tactics_hanging_node = pd.DataFrame(Y_test_tactics_hanging_node, columns=Y_test_tactic.columns)
Y_test_techniques_hanging_node = pd.DataFrame(Y_test_techniques_hanging_node, columns=Y_test.columns)

(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], [])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] []
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1109', 'T1500'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1109', 'T1500']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1082', 'T1059', 'T1060'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1082', 'T1059', 'T1060']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA

(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1038'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1038']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1076', 'T1087', 'T1064', 'T1060', 'T1078', 'T1068', 'T1002', 'T1018', 'T1046', 'T1086'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1076', 'T1087', 'T1064', 'T1060', 'T1078', 'T1068', 'T1002', 'T1018', 'T1046', 'T1086']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1113', 'T1033', 'T1082', 'T1071', 'T1064', 'T1057', 'T1056', 'T1022'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 

(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1107', 'T1064', 'T1027', 'T1132', 'T1022', 'T1105'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1107', 'T1064', 'T1027', 'T1132', 'T1022', 'T1105']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], [])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] []
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1025', 'T1120', 'T1091', 'T1092', 'T1020', 'T1039', 'T1052'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1025', 'T1120', 'T1091

(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1071', 'T1048'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1071', 'T1048']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1045', 'T1016', 'T1060', 'T1002', 'T1111'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1045', 'T1016', 'T1060', 'T1002', 'T1111']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1105'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1105']
-----------------------------
(['TA0006', 'TA0002', 'TA

(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1109', 'T1186', 'T1500'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1109', 'T1186', 'T1500']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1140', 'T1491', 'T1027', 'T1488'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1140', 'T1491', 'T1027', 'T1488']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1171'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1171']
-----------------------------
(['TA0006', 'TA0002', 'TA

...

(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1117', 'T1086'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1117', 'T1086']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], [])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] []
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1021', 'T1175'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1021', 'T1175']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', '

(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1193', 'T1070', 'T1203', 'T1086'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1193', 'T1070', 'T1203', 'T1086']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], [])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] []
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1047', 'T1113', 'T1033', 'T1082', 'T1071', 'T1036', 'T1016', 'T1083', 'T1074', 'T1027', 'T1132', 'T1105'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1047', 'T1113

(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1205'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1205']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1491'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1491']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1055'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1055']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA

(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1123', 'T1115', 'T1021', 'T1125', 'T1057'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1123', 'T1115', 'T1021', 'T1125', 'T1057']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1119', 'T1082', 'T1071', 'T1106', 'T1005', 'T1107', 'T1087', 'T1059', 'T1083', 'T1064', 'T1057', 'T1041', 'T1203', 'T1132'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1119', 'T1082', 'T1071', 'T1106', 'T1005', 'T1107', 'T1087', 'T1059', 'T1083', 'T1064', 'T1057', 'T1041', 'T1203', 'T1132']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'T

-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1063', 'T1117', 'T1064', 'T1060', 'T1027', 'T1105'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1063', 'T1117', 'T1064', 'T1060', 'T1027', 'T1105']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1108'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1108']
-----------------------------
(['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'], ['T1083', 'T1057', 'T1105'])
['TA0006', 'TA0002', 'TA0040', 'TA0003', 'TA0004', 'TA0008', 'TA0005', 'TA0010', 'TA0007', 'TA0009', 'TA0011', 'TA0001'] ['T1083', 'T1057'

In [17]:
evaluation(Y_test_tactics_hanging_node, Y_test_tactic)

,metric,result
0,macro precision,0.231104
1,micro precision,0.231104
2,macro recall,1.000000
3,micro recall,1.000000
4,macro fscore,0.269211
5,micro fscore,0.273101


In [18]:
evaluation(Y_test_techniques_hanging_node, Y_test)

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,metric,result
0,macro precision,0.600250
1,micro precision,0.488156
2,macro recall,0.390219
3,micro recall,0.331800
4,macro fscore,0.480735
5,micro fscore,0.446111


# Train Model on Reports Tactics:

In [ ]:
# see if training on only techniques improves or not
# only train techniques on reports belonging to a techniques' tactic

In [19]:
# Key as technique, value as tactic.

from collections import defaultdict
tactic_to_technique = defaultdict(list)
for tactic in tactic_data:
    for technique in tactic_data[tactic]['Technique_ID'][0]:
        if tactic in Y_train_tactic.columns:
            tactic_to_technique[technique].append(tactic)

In [20]:
Y = pd.concat([Y_train, Y_test]).reset_index(drop=True)
Y_tactic = pd.concat([Y_train_tactic, Y_test_tactic]).reset_index(drop=True)

In [21]:
# check for matching between tactic and technique
# find technique that are not in dictionary. 
# for Y_r, find report that only contains that technique and find what tactics is has 

for technique in Y.columns:
    if technique not in tactic_to_technique:
        index = Y[(Y.sum(axis=1)==1) & (Y[technique] == 1)].index[-1]
        tactics = [tactic for tactic in Y_tactic.columns if 
                       int(Y_tactic.loc[index, tactic])==1]
        if len(tactics) == 0:
            print(technique)
            print(Y[(Y.sum(axis=1)==1) & (Y[technique] == 1)].index)
        tactic_to_technique[technique] = tactics
    
    

In [22]:
Y_train_tactic[Y_train_tactic.sum(axis=1)==0]

,TA0006,TA0002,TA0040,TA0003,TA0004,TA0008,TA0005,TA0010,TA0007,TA0009,TA0011,TA0001
1041,0,0,0,0,0,0,0,0,0,0,0,0
1154,0,0,0,0,0,0,0,0,0,0,0,0
1216,0,0,0,0,0,0,0,0,0,0,0,0
1333,0,0,0,0,0,0,0,0,0,0,0,0
1334,0,0,0,0,0,0,0,0,0,0,0,0
1335,0,0,0,0,0,0,0,0,0,0,0,0
1337,0,0,0,0,0,0,0,0,0,0,0,0
1416,0,0,0,0,0,0,0,0,0,0,0,0
1417,0,0,0,0,0,0,0,0,0,0,0,0
1418,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
Y_train = Y_train.reset_index(drop=True)
Y_train_tactic = Y_train_tactic.reset_index(drop=True)


In [24]:
classifiers = []
for technique in Y_train.columns:
    indexes = Y_train_tactic[Y_train_tactic.apply(lambda x: any(x[tactic] ==1 for tactic in tactic_to_technique[technique]), axis=1)].index
    X_train_technique = X_train.loc[indexes]
    Y_train_technique = Y_train.loc[indexes, technique]
    clf = LinearSVC(penalty = 'l2', loss = 'squared_hinge', dual = False, max_iter = 1000, class_weight = 'balanced', random_state=42)
    clf.fit(X_train_technique, Y_train_technique)
    classifiers.append(clf)
    

In [25]:
Y_pred = pd.DataFrame(0, index=X_test.index, columns=Y_test.columns)
for i, technique in enumerate(Y_test.columns):
    indexes = Y_test_pred_tactic[Y_test_pred_tactic.apply(lambda x: any(x[tactic] ==1 for tactic in tactic_to_technique[technique]), axis=1)].index
    Y_pred.loc[indexes, technique] = classifiers[i].predict(X_test.loc[indexes])

In [26]:
evaluation(Y_pred, Y_test) # performance for technique 

/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/vol/bitbucket/lgf21/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


,metric,result
0,macro precision,0.514669
1,micro precision,0.530709
2,macro recall,0.334334
3,micro recall,0.387579
4,macro fscore,0.438668
5,micro fscore,0.494207
